### Setup the noisy simulator according to hardware calibration data

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService


# get a real backend from the runtime service
service = QiskitRuntimeService(channel="ibm_quantum", 
                               token="eff90e8bee70d824d8e6c145072c95ec9ba28369fd0c8aa5b192380f5c7ffc207f64bab13f2ae8897269ecdfa88b1f288c4e58c57bdcf86ed8bd500257170657", 
                               instance="ibm-q-ornl/ornl/csc509")

/var/folders/3t/7ts1c6s95hq650lfkvk66tkh0000gn/T/ipykernel_4920/655742877.py:5: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum",


In [2]:
service.backends()

[<IBMBackend('ibm_strasbourg')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibm_brussels')>,
 <IBMBackend('ibm_marrakesh')>,
 <IBMBackend('ibm_aachen')>,
 <IBMBackend('ibm_kingston')>,
 <IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_torino')>,
 <IBMBackend('ibm_fez')>]

Then, one can grab the desired backend.

In [2]:
device = service.backend('ibm_fez')

We can setup the noisy simulator by passing the backend properties to the aer simulator.

In [3]:
from qiskit_aer import AerSimulator

backend_sim = AerSimulator.from_backend(device)

---

In [6]:
import time
import numpy as np
import random
import dill as pickle

import qiskit
from qiskit.circuit.library import RealAmplitudes
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector, state_fidelity
import hadamard_random_forest as hrf

shots = 10000
num_trees = 111
num_qubits = 2
num_layers = 4
num_trials = 1
fid_sv_list = []
quantum_time_list = []
post_time_list = []
hf_sim = {}

with open('params_0402.pkl', 'rb') as f:
    loaded_param = pickle.load(f)

hrf.fix_random_seed(999)
print("=================================== Simulation Start =============================================")
for i in range(num_trials):
    print(f"=================================== Simulate {i}-th sample =========================================")
    ansatz = RealAmplitudes(num_qubits, entanglement='pairwise', reps=num_layers, skip_final_rotation_layer=True)
    params = loaded_param['parameter'][(num_qubits, num_layers)][0]
    list_circuits = hrf.get_circuits(num_qubits, ansatz)
    
    # record quantum time
    start_time = time.time()
    samples = hrf.get_samples_noisy(num_qubits, list_circuits, shots, params, backend_sim, error_mitigation=True)
    end_time = time.time()
    elapsed_time1 = end_time - start_time
    print("Quantum time for sampling:", elapsed_time1)
    quantum_time_list.append(elapsed_time1)

    # record post-processing time
    start_time = time.time()
    noisy_job_samp = hrf.get_statevector(num_qubits, num_trees, samples, save_tree=True)
    end_time = time.time()
    elapsed_time2 = end_time - start_time
    print("Post-processing time for getting the signs:", elapsed_time2)
    post_time_list.append(elapsed_time2)
    
    noisy_job_samp = noisy_job_samp/np.linalg.norm(noisy_job_samp)

    # Bind parameters
    ansatz.assign_parameters(params, inplace=True)
    statevector = Statevector(ansatz)
    print(statevector[0])
    
    fid_noisy = state_fidelity(noisy_job_samp, statevector)
    fid_sv_list.append(fid_noisy)
    print("HForest FakeIBMQ Fidelity:", fid_noisy)
    print("Fidelity Upper Bound:", state_fidelity(np.abs(noisy_job_samp), np.abs(statevector)))
    negative_count = np.sum(statevector.data.real < 0)
    print(f"Qubits: {num_qubits} Layers:{ num_layers} positive_count: {100*np.sum(statevector.data.real[(statevector.data.real > 0)]**2)}%")
    print(f"Qubits: {num_qubits} Layers:{ num_layers} negative_count: {100*np.sum(statevector.data.real[(statevector.data.real < 0)]**2)}%")

    # Store data
    hf_sim.update({i:{
        "n_qubits": num_qubits,
        "n_shots": shots,
        "n_layers": num_layers,
        "n_trees": num_trees,
        "fid_state": fid_noisy,
        "time_quanutm": elapsed_time1,
        "time_postprocess": elapsed_time2,
        "samples": samples,
        "state": statevector,
        "state_hardware": noisy_job_samp,
        "fid_bound": state_fidelity(np.abs(noisy_job_samp), np.abs(statevector)),
        "negative_count": negative_count,
        "negative_percentage": 100*np.sum(statevector.data.real[(statevector.data.real < 0)]**2)}})

print("=================================== Simulation End =============================================")

=================================== Simulation Start =============================================
=================================== Simulate 0-th sample =========================================
=========== New M3 calibration detected ===========
=========== New M3 calibration detected ===========
Quantum time for sampling: 10.916259050369263
Post-processing time for getting the signs: 3.9344239234924316
(0.7034727238789156+0j)
HForest FakeIBMQ Fidelity: 0.9999571694159465
Fidelity Upper Bound: 0.9999571694159465
Qubits: 2 Layers:4 positive_count: 94.31613524219738%
Qubits: 2 Layers:4 negative_count: 5.683864757802615%
=================================== Simulation End =============================================


In [ ]:
import hadamard_random_forest as hrf

hrf.seed_everything(999)
tree, spanning = hrf.generate_hypercube_tree(num_qubits=5)
samples = hrf.get_samples_noisy(
    num_qubits=5,
    sampler=my_sampler,
    circuits=my_circuits,
    shots=1024,
    parameters=params,
    backend_sim=my_backend,
    error_mitigation=True
)
state = hrf.get_statevector(num_qubits=5, num_trees=20, samples=samples)

In [8]:
!pip show hadamard_random_forest

Name: hadamard-random-forest
Version: 0.1.0
Summary: Hadamard Random Forest: real valued quantum state reconstruction
Home-page: 
Author: Zhixin (Jack) Song
Author-email: zsong300@gatech.edu
License: 
Location: /Users/jacksong/Documents/GitHub/Quantum-HRF-Tomography
Editable project location: /Users/jacksong/Documents/GitHub/Quantum-HRF-Tomography
Requires: matplotlib, mthree, networkx, numpy, pygraphviz, qiskit, qiskit-aer, qiskit-experiments, qiskit-ibm-runtime, scipy, tqdm, treelib
Required-by: 
